In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [2]:
data = [train_data, test_data]
for df in data:
    if 'label' in df.columns:
        df.drop('label', axis=1)
    df = df/255
print(f"data shape in train {train_data.shape}\ndata shape in test {test_data.shape}")

data shape in train (42000, 785)
data shape in test (28000, 784)


In [3]:
print(train_data['label'].value_counts())

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data.drop('label', axis=1),train_data['label'],test_size=0.33, random_state=24)

In [5]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(28140, 784) (13860, 784) (28140,) (13860,)


In [6]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
X_test_r = X_test.values.reshape(13860, 28,  28, 1)
X_train_r = X_train.values.reshape(28140, 28,  28, 1)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout, Conv2D, MaxPooling2D, Flatten, GaussianNoise, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.random import set_seed
from tensorflow.keras.optimizers import SGD

model_conv2D = Sequential()
model_conv2D.add(Conv2D(32, (5, 5), activation='relu',
 input_shape=(28, 28, 1)))
model_conv2D.add(MaxPooling2D((3, 3)))

model_conv2D.add(Conv2D(64, (3, 3), activation='relu'))

model_conv2D.add(Flatten())
model_conv2D.add(Dense(64, activation='relu'))
model_conv2D.add(Dense(10, activation='softmax'))


es = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)]

model_conv2D.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy'
)

model_conv2D.fit(
    x = X_train_r,
    y = y_train,
    epochs = 10,
    callbacks=[es],
    validation_data=(X_test_r, y_test)
)


Epoch 1/10
880/880 [==============================] - 13s 13ms/step - loss: 2.3789 - val_loss: 0.1379
Epoch 2/10
880/880 [==============================] - 11s 12ms/step - loss: 0.1085 - val_loss: 0.1087
Epoch 3/10
880/880 [==============================] - 11s 12ms/step - loss: 0.0778 - val_loss: 0.1012
Epoch 4/10
880/880 [==============================] - 11s 13ms/step - loss: 0.0687 - val_loss: 0.1889
Epoch 5/10
880/880 [==============================] - 11s 12ms/step - loss: 0.0601 - val_loss: 0.1068
Epoch 6/10
880/880 [==============================] - 11s 12ms/step - loss: 0.0564 - val_loss: 0.1043
Epoch 00006: early stopping


In [9]:
from sklearn.metrics import accuracy_score
preds = model_conv2D.predict_classes(X_test_r)
y_test_x = [np.argmax(r) for r in y_test]
print(f'Accuracy: {round(accuracy_score(y_test_x, preds), 5)}')

Accuracy: 0.9816


In [10]:
test = pd.Series(model_conv2D.predict_classes(test_data.values.reshape(28000, 28, 28, 1)))
test_subm = pd.DataFrame(data = {'ImageId': list(range(1,28001)), 'Label':test})
test_subm.to_csv('test_subm.csv', index=False)